## Problem Statment
A Company wants to automate the loan eligibility process based on customer details provided while filling online application form. The details filled by the customer are Gender, Marital Status, Education, Number of Dependents, Income of self and co applicant, Required Loan Amount, Required Loan Term, Credit History and others. The requirements are as follows:

1.)Check eligibility of the Customer given the inputs described above.(Classification)

In [1]:
import pandas as pd

In [3]:
trd = pd.read_csv(r"D:\ENGINEERING LIFE\Computer Languages\Data Science\Wakad Lectures\39th Lecture 05_03_2023\Project\training_set.csv")
tsd = pd.read_csv(r"D:\ENGINEERING LIFE\Computer Languages\Data Science\Wakad Lectures\39th Lecture 05_03_2023\Project\testing_set.csv")

In [4]:
trd.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.0,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,NaN,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
tsd.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


## Missing Value Treatment

In [6]:
trd.isna().sum()

Loan_ID               0
Gender               15
Married               3
Dependents           15
Education             1
Self_Employed        32
ApplicantIncome       2
CoapplicantIncome     1
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [7]:
tsd.isna().sum()

Loan_ID               0
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

In [8]:
for i in tsd.columns:
    if(trd[i].dtypes == "object"):
        x = trd[i].mode()[0]
        trd[i] = trd[i].fillna(x)
        tsd[i] = tsd[i].fillna(x)
    else:
        x = trd[i].mean()
        trd[i] = trd[i].fillna(x)
        tsd[i] = tsd[i].fillna(x)

In [9]:
trd.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [10]:
tsd.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

## Outliers Treatment

In [11]:
cat = []
con = []
for i in trd.columns:
    if(trd[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [12]:
cat

['Loan_ID',
 'Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Loan_Status']

In [13]:
con

['ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History']

In [14]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
import pandas as pd
trdconss = pd.DataFrame(ss.fit_transform(trd[con]),columns=con)

In [15]:
outliers = []
for i in trdconss.columns:
    outliers.extend(list(trdconss[(trdconss[i]<-3)|(trdconss[i]>3)].index))
    
from numpy import unique
outliers = unique(outliers)

In [16]:
trd = trd.drop(index = outliers,axis=0)

In [17]:
trd.shape

(577, 13)

In [18]:
trd.index = range(0,trd.shape[0])

## EDA

In [19]:
trd[con].skew()

ApplicantIncome      2.145688
CoapplicantIncome    1.351359
LoanAmount           1.113132
Loan_Amount_Term    -2.077031
Credit_History      -1.921860
dtype: float64

In [20]:
def ANOVA(df,cat,con):
    from statsmodels.formula.api import ols
    from statsmodels.stats.anova import anova_lm
    rel = con + " ~ " + cat
    model = ols(rel,df).fit()
    Q = anova_lm(model)
    pval = round(Q.iloc[0:1,4][0],4)
    return pval

In [21]:
for i in con:
    p = ANOVA(trd,"Loan_Status",i)
    print("Loan_Status vs",i," Pval ---->",p)

Loan_Status vs ApplicantIncome  Pval ----> 0.8943
Loan_Status vs CoapplicantIncome  Pval ----> 0.2821
Loan_Status vs LoanAmount  Pval ----> 0.3975
Loan_Status vs Loan_Amount_Term  Pval ----> 0.6636
Loan_Status vs Credit_History  Pval ----> 0.0


## Preprocessing

In [22]:
X_trd = trd.drop(labels=["Loan_ID","Loan_Status"],axis=1)
Y_trd = trd[["Loan_Status"]]

cat = []
con = []
for i in X_trd.columns:
    if(X_trd[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
x1_trd = pd.DataFrame(mm.fit_transform(X_trd[con]),columns=con)
x2_trd = pd.get_dummies(X_trd[cat])
Xnew_trd = x1_trd.join(x2_trd)

In [23]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew_trd,Y_trd,test_size=0.2,random_state=21)


## Logistic Regression

In [24]:
from warnings import filterwarnings
filterwarnings("ignore")

In [25]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
model = lr.fit(xtrain,ytrain)
tr_pred = model.predict(xtrain)
ts_pred = model.predict(xtest)
from sklearn.metrics import accuracy_score
tr_acc = accuracy_score(ytrain,tr_pred)
ts_acc = accuracy_score(ytest,ts_pred)

In [26]:
tr_acc

0.8156182212581344

In [27]:
ts_acc

0.8448275862068966

## Decision Tree

In [28]:
def modeller(mo):
    model = mo.fit(xtrain,ytrain)
    tr_pred = model.predict(xtrain)
    ts_pred = model.predict(xtest)
    from sklearn.metrics import accuracy_score
    tr_acc = round(accuracy_score(ytrain,tr_pred),2)
    ts_acc = round(accuracy_score(ytest,ts_pred),2)
    return tr_acc,ts_acc

In [29]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=21)
modeller(dtc)

(1.0, 0.68)

In [30]:
for i in range(2,20):
    dtc = DecisionTreeClassifier(random_state=21,max_depth=i)
    print(i,modeller(dtc))

2 (0.81, 0.84)
3 (0.83, 0.83)
4 (0.84, 0.81)
5 (0.85, 0.8)
6 (0.87, 0.8)
7 (0.9, 0.77)
8 (0.92, 0.73)
9 (0.95, 0.71)
10 (0.97, 0.69)
11 (0.98, 0.7)
12 (0.99, 0.7)
13 (1.0, 0.7)
14 (1.0, 0.68)
15 (1.0, 0.68)
16 (1.0, 0.68)
17 (1.0, 0.68)
18 (1.0, 0.68)
19 (1.0, 0.68)


## KNN

In [31]:
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier()
modeller(knc)

(0.81, 0.75)

In [32]:
for i in range(2,20):
    knc = KNeighborsClassifier(n_neighbors=i)
    print(i,modeller(knc))

2 (0.9, 0.69)
3 (0.85, 0.75)
4 (0.86, 0.72)
5 (0.81, 0.75)
6 (0.8, 0.72)
7 (0.77, 0.7)
8 (0.79, 0.71)
9 (0.76, 0.73)
10 (0.76, 0.72)
11 (0.75, 0.74)
12 (0.76, 0.74)
13 (0.74, 0.75)
14 (0.74, 0.76)
15 (0.74, 0.77)
16 (0.74, 0.76)
17 (0.72, 0.77)
18 (0.73, 0.76)
19 (0.72, 0.77)


## Training the model with entire data in training set

In [33]:
dtc = DecisionTreeClassifier(random_state=21,max_depth=3)
finmodel = dtc.fit(Xnew_trd,Y_trd)

## Data preparation for testing set

In [34]:
xtrain.columns

Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Gender_Female', 'Gender_Male',
       'Married_No', 'Married_Yes', 'Dependents_0', 'Dependents_1',
       'Dependents_2', 'Dependents_3+', 'Education_Graduate',
       'Education_Not Graduate', 'Self_Employed_No', 'Self_Employed_Yes',
       'Property_Area_Rural', 'Property_Area_Semiurban',
       'Property_Area_Urban'],
      dtype='object')

In [35]:
X_tsd = tsd.drop(labels=["Loan_ID"],axis=1)

cat = []
con = []
for i in X_tsd.columns:
    if(X_tsd[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [36]:
X1_tsd = pd.DataFrame(mm.transform(X_tsd[con]),columns=con)
X2_tsd = pd.get_dummies(X_tsd[cat])
Xnew_tsd = X1_tsd.join(X2_tsd)

In [37]:
Xnew_tsd[Xnew_trd.columns]

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,0.271482,0.000000,0.272237,0.6,1.000000,0,1,0,1,1,0,0,0,1,0,1,0,0,0,1
1,0.142613,0.167038,0.315364,0.6,1.000000,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1
2,0.236389,0.200445,0.536388,0.6,1.000000,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1
3,0.106741,0.283519,0.245283,0.6,0.842199,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1
4,0.152361,0.000000,0.185984,0.6,1.000000,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,0.188088,0.197884,0.280323,0.6,1.000000,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1
363,0.195350,0.078953,0.285714,0.6,1.000000,0,1,0,1,1,0,0,0,1,0,1,0,0,0,1
364,0.151094,0.221938,0.315364,0.6,0.842199,0,1,1,0,1,0,0,0,1,0,1,0,0,1,0
365,0.236389,0.266481,0.401617,0.6,1.000000,0,1,0,1,1,0,0,0,1,0,1,0,1,0,0


In [38]:
pred_LS = finmodel.predict(Xnew_tsd)

In [39]:
pred_LS

array(['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y

## Final DF

In [40]:
Q = tsd[["Loan_ID"]]
Q["Loan_Status"]=pred_LS

In [41]:
Q.to_csv("Loan Eligibilty and Approval")

In [42]:
Q.head()

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,Y
